In [2]:
import polars as pl

In [3]:
tickets_historicos = pl.read_csv(
    source = './Tickets/Tickets Historico.txt',
    separator = ';',
    columns = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion', 'Fecha Termino', 'Fecha Cierre'],
    try_parse_dates = True
)

In [4]:
tickets_historicos = tickets_historicos.rename({ 'Numero Ticket': 'TicketID'})

In [5]:
tickets_actuales = pl.read_csv(
    source = './Tickets/Tickets Actual.csv',
    separator = '|',
    columns = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion', 'Fecha Termino', 'Fecha Cierre'],
)

In [6]:
tickets_actuales = tickets_actuales.rename({ 'Numero Ticket': 'TicketID'})

In [7]:
tickets_actuales = tickets_actuales.select(
    'TicketID', 'Ubicacion', 'Service Desk', 'Estado',
    pl.col('Fecha Creacion').cast(pl.Date),
    pl.col('Fecha Termino').str.to_date('%d/%m/%Y'),
    pl.col('Fecha Cierre').str.to_date('%d/%m/%Y')    
)

In [8]:
tickets_actuales = tickets_actuales.filter(
    pl.col('TicketID').str.starts_with('WO')
)

In [9]:
tickets = pl.concat([tickets_historicos, tickets_actuales], how='vertical')

In [11]:
tickets = tickets.unique(subset='TicketID')

In [12]:
tickets = tickets.sort(
    by=['TicketID', 'Fecha Creacion']
).unique(
    subset='TicketID',
    keep='last',
    maintain_order=True
)

In [13]:
tickets = tickets.with_columns(
    pl.col('Ubicacion').str.split_exact(' - ', 1)
    .struct.rename_fields(['Agencia', 'AgenciaID'])
).unnest('Ubicacion').cast({'AgenciaID': pl.Int64})

In [15]:
tickets = tickets.with_columns(
    pl.when(pl.col('Fecha Termino').is_null())
        .then(pl.col('Fecha Cierre'))
        .otherwise(pl.col('Fecha Termino'))
        .alias('Fecha Real Fin')
)

In [17]:
tickets = tickets.with_columns(
    (pl.col('Fecha Real Fin') - pl.col('Fecha Creacion'))
    .dt.total_days()
    .alias('Dias Cierre')
)

In [23]:
tickets = tickets.with_columns(
    pl.when(pl.col('Dias Cierre').is_null()).then(None)
    .when(pl.col('Dias Cierre') < 3).then(pl.lit('0 a 3 dias'))
    .when(pl.col('Dias Cierre') < 7).then(pl.lit('3 a 7 dias'))
    .when(pl.col('Dias Cierre') < 15).then(pl.lit('8 a 15 dias'))
    .otherwise(pl.lit('+15 dias'))
    .alias('Grupo Dias')
)

In [50]:
from pathlib import Path

In [52]:
archivos = list(Path('./Atenciones/').glob('*.xlsx'))

In [58]:
atenciones = pl.DataFrame()

for archivo in archivos:
    file = archivo.name
    filePath = f'./Atenciones/{file}'
    data = pl.read_excel(
                source=filePath,
                sheet_name='Hoja1',
                engine='xlsx2csv',
                infer_schema_length=0,
                read_options={
                    'columns': ['Numero Ticket', 'Tipo de Ticket', 'Proveedor', 'Costo Atencion']
                }
            )
    atenciones = pl.concat([atenciones, data], how='vertical')
    

In [60]:
atenciones = atenciones.rename({ 'Numero Ticket': 'TicketID'})

In [63]:
atenciones = atenciones.select(
    pl.col(['TicketID', 'Tipo de Ticket', 'Proveedor']),
    pl.col('Costo Atencion').str.replace(',', '.')
    .str.to_uppercase()
    .str.strip_chars()
    .str.replace_many(
        ['COSTO CERO', 'SIN COSTO'], '0'
    )
    .alias('Costo Atencion')
)

In [64]:
def text_to_decimal(valor:str):
    try:
        return float(valor)
    except:
        return None

In [84]:
atenciones = atenciones.with_columns(
    pl.col('Costo Atencion').map_elements(text_to_decimal).alias('Costo')
)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11896\3495111518.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  atenciones = atenciones.with_columns(


In [87]:
baseTotal = tickets.join(
    other = atenciones,
    on = 'TicketID',
    how = 'inner' 
).select(
    pl.col(['TicketID', 'AgenciaID', 'Agencia', 'Service Desk', 'Estado', 'Fecha Creacion', 'Grupo Dias', 'Costo']),
    pl.col('Fecha Real Fin').alias('Fecha Cierre'),
    pl.col('Tipo de Ticket').alias('Tipo Ticket'),
)

In [90]:
baseTotal.write_excel(
    workbook='Consolidado.xlsx',    
    autofit=True,
    dtype_formats={pl.Date: 'dd/mm/yyyy'},
    float_precision=2,
    table_style='Table Style Medium 4'
)